In [ ]:
import sys
print(sys.version, sys.platform, sys.executable)

import math
import time

import torch.nn as nn
from torch.nn.modules.utils import _triple

from torchsummary import summary
from torch import nn, optim

import cv2
import numpy as np
import torch
import torch.nn.functional as F

from torchviz import make_dot
import matplotlib.pyplot as plt
from matplotlib import *
import pylab

import os
from os import listdir
from os.path import isfile, join, isdir
import gc

from pathlib import Path

import cv2
import numpy as np
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

torch.cuda.empty_cache()
gc.collect()

# Crezione dataset

In [ ]:
#Proprietà input e device:

CLIP_LEN, RESIZE_HEIGHT, CROP_SIZE = 10, 128, 112
resize_height = RESIZE_HEIGHT
crop_size = CROP_SIZE
clip_len = CLIP_LEN

device = torch.device('cpu')#('cuda:0' if torch.cuda.is_available() else 'cpu')

def center_crop(image):
        height_index = math.floor((image.shape[0] - crop_size) / 2)
        width_index = math.floor((image.shape[1] - crop_size) / 2)
        image = image[height_index:height_index + crop_size, width_index:width_index + crop_size, :]
        return np.array(image).astype(np.uint8)

In [ ]:
class Creazione_VideoTestset(Dataset):

    def __init__(self, directory, mode='train', clip_len=8, num_sec=8, cond=""):
        folder = Path(directory)
        self.type = cond
        self.clip_len = clip_len-1
        
        self.num_sec=num_sec
        
        # the following three parameters are chosen as described in the paper section 4.1
        self.resize_height = 128/2  
        self.resize_width = 171/2
        self.crop_size = 112/2

        # obtain all the filenames of files inside all the class folders 
        # going through each class folder one at a time
        self.fnames, labels = [], []
        for label in sorted(os.listdir(folder)):
            for fname in os.listdir(os.path.join(folder, label)):
                self.fnames.append(os.path.join(folder, label, fname))
                labels.append(label)
                
        # prepare a mapping between the label names (strings) and indices (ints)
        self.label2index = {label:index for index, label in enumerate(sorted(set(labels)))} 
        
        # convert the list of label names into an array of label indices
        self.label_array = np.array([self.label2index[label] for label in labels], dtype=int)        
        #create_dataset()
        
    def create_dataset(self):
        count = 0
        clips = []
        clip = []
        prev_label = -1
        os.mkdir('testset'+ self.type +'_'+"num_sec_"+str(self.num_sec)+"_FPS_"+str((self.clip_len+1)/self.num_sec))
        
        for i in np.arange(len(self.fnames)):
            if(self.label_array[i] != prev_label):
                count = 0
                os.mkdir('testset'+ self.type +'_'+"num_sec_"+str(self.num_sec)+"_FPS_"+str((self.clip_len+1)/self.num_sec)+'/'+str(self.label_array[i]))
            os.mkdir('testset'+ self.type +'_'+"num_sec_"+str(self.num_sec)+"_FPS_"+str((self.clip_len+1)/self.num_sec)+'/'+str(self.label_array[i])+'/'+str(count))
            clip = self.loadvideo(self.fnames[i], i, count)
            
            count = count + 1
            prev_label = self.label_array[i]
    
    def __getitem__(self, index):
        buffer = []
        # loading and preprocessing. TODO move them to transform classes
        buffer = self.loadvideo(self.fnames[index])
        control = buffer != []
        if control:
            buffer = self.normalize(buffer)
            return buffer[0,:,:,:,:], self.label_array[index-1]    
        return buffer, self.label_array[index]
        
    def loadvideo(self, fname, index, count):
        self.resize_height = int(128*3/2)
        self.resize_width = int(171*3/2)
        self.crop_size = int(112*3/2)
        
        cap, retaining, clips = cv2.VideoCapture(fname), True, []
        
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(frame_count)
        frame_count = self.num_sec*frame_count/8
        buffer = []
        i = -1
        frame_collect = 0
        if(self.clip_len != 0):
            counter = counter = (int)(frame_count/self.clip_len)
            
            if(counter < 2):
                counter = 1
        while retaining:
            
            i = i+1
            retaining, frame = cap.read()
            
            if not retaining and frame is None:

                continue
            if(i == 0 and self.clip_len == 0):
                resize_width = math.floor(frame.shape[1] / frame.shape[0] * self.resize_height)
                # make sure it can be cropped correctly
                control3 = self.resize_width < self.crop_size
                if control3:
                    self.resize_width = self.resize_height
                    self.resize_height = math.floor(frame.shape[0] / frame.shape[1] * self.resize_width)
                    
                tmp = center_crop(cv2.resize(frame, (self.resize_width, self.resize_height)))
               
                
                cv2.imwrite('testset'+ self.type +'_'+"num_sec_"+str(self.num_sec)+"_FPS_"+str((self.clip_len+1)/self.num_sec)+'/'+str(self.label_array[index])+'/'+str(count)+"/"+str(self.label_array[index])+'_'+str(frame_collect)+'.jpg', tmp)
                
                cap.release()
                return buffer
            
            control = ((i % counter == 0) and frame_collect<=self.clip_len) #and (np.shape(buffer))

            #if(buffer)
            if control:
                frame_collect = frame_collect + 1
                resize_width = math.floor(frame.shape[1] / frame.shape[0] * self.resize_height)
                # make sure it can be cropped correctly
                control3 = self.resize_width < self.crop_size
                if control3:
                    self.resize_width = self.resize_height
                    self.resize_height = math.floor(frame.shape[0] / frame.shape[1] * self.resize_width)
                    
                tmp = center_crop(cv2.resize(frame, (self.resize_width, self.resize_height)))
                
                cv2.imwrite('testset'+ self.type +'_'+"num_sec_"+str(self.num_sec)+"_FPS_"+str((self.clip_len+1)/self.num_sec)+'/'+str(self.label_array[index])+'/'+str(count)+"/"+str(self.label_array[index])+'_'+str(frame_collect)+'.jpg', tmp)
            float_control = False
            control2 = (i == frame_count-1 and float_control == True and frame_collect == self.clip_len )
            if control2:
                frame_collect = frame_collect + 1
                resize_width = math.floor(frame.shape[1] / frame.shape[0] * self.resize_height)
                # make sure it can be cropped correctly
                control3 = self.resize_width < self.crop_size
                if control3:
                    self.resize_width = self.resize_height
                    self.resize_height = math.floor(frame.shape[0] / frame.shape[1] * self.resize_width)
                    
                tmp = center_crop(cv2.resize(frame, (self.resize_width, self.resize_height)))
                cv2.imwrite('testset'+ self.type +'_'+"num_sec_"+str(self.num_sec)+"_FPS_"+str((self.clip_len+1)/self.num_sec)+'/'+str(self.label_array[index])+'/'+str(count)+"/"+str(self.label_array[index])+'_'+str(frame_collect)+'.jpg', tmp)
                 
        cap.release()
        print('testset'+ self.type +'_'+"num_sec_"+str(self.num_sec)+"_FPS_"+str((self.clip_len+1)/self.num_sec)+'/'+str(self.label_array[index])+'/'+str(count)+"/"+str(self.label_array[index])+'_'+str(frame_collect)+'.jpg')
        return []
    
    def normalize(self, buffer):
        # Normalize the buffer>
        # NOTE: Default values of RGB images normalization are used, as precomputed 
        # mean and std_dev values (akin to ImageNet) were unavailable for Kinetics. Feel 
        # free to push to and edit this section to replace them if found. 
        buffer = (buffer - 128)/128
        return buffer

    def __len__(self):
        return len(self.fnames)

In [ ]:
pippo = np.arange(22)+1
minni = np.arange(8)+1
for i in pippo:
    for j in minni:
        p = i * j
        if(p == 8 or p == 16 or p == 24 or p == 32 or p == 40 or p == 48 or p == 56 or p == 64 or p == 72):
                x = Creazione_VideoTestset("TestSet/Dataset_Day", clip_len=j*i, num_sec = j, cond="Day")
                x.create_dataset()
            